In [96]:
import pymysql
import json
from functools import partial
import csv
import networkx as nx
import freeman as fm

def run_db_query(connection, query, args=None):
    with connection.cursor() as cursor:
        print('Executando query:')
        cursor.execute(query, args)
        for result in cursor:
            print(result)
            
def return_db_results(connection, query, args=None):#Gambiarra para pegar o id
    with connection.cursor() as cursor:
        cursor.execute(query, args)
        for result in cursor:
            return result #retorna logo o primeiro

def multiple_db_results(connection, query, args=None):#Gambiarra para pegar o id
    with connection.cursor() as cursor:
        cursor.execute(query, args)
        res = cursor.fetchall()
        return res

connection = pymysql.connect(
    host='localhost',
    user='root',
    password='vlm1998',
    database='redes')

db = partial(run_db_query, connection)

In [140]:
def add_node(g, n):
    g.add_node(n)
    g.nodes[n]['label'] = n
    g.nodes[n]['color'] = (255, 0, 0)

def add_edge(g, n, m):
    print("De: ", n, "para: ", m)
    g.add_edge(n, m)

def get_team_name(connection, id_team):
    with connection.cursor() as cursor:
        cursor.execute("SELECT nome FROM times WHERE id_time = %s", (id_team))
        res = cursor.fetchone()
        if res:
            return res
        else:
            return -1
def list_all_teams(connection):
    with connection.cursor() as cursor:
        cursor.execute("SELECT nome FROM times")
        res = cursor.fetchall()
        return res
def list_all_transfers_2019(connection):
    with connection.cursor() as cursor:
        cursor.execute("SELECT * FROM transferencia WHERE ano = '2019' ")
        res = cursor.fetchall()
        return res
def get_from_where_2019(connection, id_jogador):
    with connection.cursor() as cursor:
        cursor.execute("SELECT times.nome FROM waived INNER JOIN times USING(id_time) WHERE id_jogador = %s AND ano = '2019' ", (id_jogador))
        res = cursor.fetchone()
        if res:
            return res
        else:
            return -1


In [144]:
a = fm.Animation()
g = fm.Graph(nx.Graph())
a.rec(g)

transfers = list_all_transfers_2019(connection)
teams = list_all_teams(connection)

for team in teams:
    add_node(g, team)
    
#Existem casos em que um time assina com um jogador e na mesma temporada vende ele, vamos ignorar estes casos por enquanto
for transfer in transfers:
    antigo = get_from_where_2019(connection, transfer[1])
    novo = get_team_name(connection, transfer[2])
    if(antigo != -1) and (novo != -1):
        if(antigo != novo):
            add_edge(g, antigo, novo)

a.rec(g)
a.play()

De:  ('Miami Heat',) para:  ('Houston Rockets',)
De:  ('Golden State Warriors',) para:  ('Sacramento Kings',)
De:  ('Cleveland Cavaliers',) para:  ('Toronto Raptors',)
De:  ('Oklahoma City Thunder',) para:  ('LA Clippers',)
De:  ('Philadelphia 76ers',) para:  ('Oklahoma City Thunder',)
De:  ('San Antonio Spurs',) para:  ('Milwaukee Bucks',)
De:  ('Phoenix Suns',) para:  ('Milwaukee Bucks',)
De:  ('Chicago Bulls',) para:  ('Cleveland Cavaliers',)
De:  ('Sacramento Kings',) para:  ('Houston Rockets',)
De:  ('Phoenix Suns',) para:  ('Chicago Bulls',)
De:  ('Phoenix Suns',) para:  ('Los Angeles Lakers',)
De:  ('New York Knicks',) para:  ('Portland Trail Blazers',)
De:  ('New York Knicks',) para:  ('Indiana Pacers',)
De:  ('Utah Jazz',) para:  ('Philadelphia 76ers',)
De:  ('Utah Jazz',) para:  ('Philadelphia 76ers',)
De:  ('Phoenix Suns',) para:  ('Detroit Pistons',)
De:  ('Memphis Grizzlies',) para:  ('Los Angeles Lakers',)
De:  ('New Orleans Pelicans',) para:  ('Oklahoma City Thunder',)
D

In [82]:
connection.close()